In [299]:
import psycopg2
from psycopg2.extensions import AsIs
import psycopg2
import pandas as pd
import numpy as np

In [300]:
%run -i ../sql_helpers.py
display(db_credentials)

{'dbname': 'gta',
 'port': 5432,
 'user': 'gta_p8',
 'password': 'r7sdkfdq',
 'host': 'ikgpgis.ethz.ch'}

In [301]:
kirchen_zuerich = pd.read_csv("csv/kirchen-zurich.csv", encoding = "utf-8", sep="	")
restaurant_zurich = pd.read_csv("csv/restaurant-zurich.csv", encoding = "utf-8", sep="	")
kirchen_zuerich.head

<bound method NDFrame.head of           @lat      @lon                                       name  \
0    47.371937  8.520866                      Synagoge Agudas Achim   
1    47.375034  8.527694          Evangelisch Methodistische Kirche   
2    47.402983  8.574152                          Kirchgemeindehaus   
3    47.369365  8.553327     Mission catholique de langue française   
4    47.369483  8.553266                 Couvent St-Nicolas de Myre   
..         ...       ...                                        ...   
115  47.332704  8.516048                            Kirche Leimbach   
116  47.365626  8.553062              Kirche Christi Wissenschafter   
117  47.414579  8.518635  Kirchliches Zentrum Neuaffoltern Glaubten   
118  47.370554  8.505807                        Thomaskirche im Gut   
119  47.360712  8.505901                                Friesenberg   

             addr:street addr:housenumber  addr:postcode addr:city building  \
0           Erikastrasse              

In [302]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [303]:
sql_string_with_placeholders = "INSERT INTO church (address_id, church_name) VALUES (%s, %s)"
cur.mogrify(sql_string_with_placeholders,(100, "kirche"))

b"INSERT INTO church (address_id, church_name) VALUES (100, 'kirche')"

In [304]:
conn.commit()
conn.close()

Kirchen

In [305]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [306]:
city_id_array = np.zeros(0)

for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #city_table
    sql_fill_city = "INSERT INTO gta_p8.city (city_postcode, city_name) SELECT '"+city_postcode+"', '"+city_name+"' WHERE NOT EXISTS (select * from gta_p8.city where city_postcode='"+city_postcode+"');"

    cur.execute(sql_fill_city, (city_postcode, city_name))

    #address_table_preparation
    cur.execute("SELECT city_id FROM gta_p8.city WHERE city_postcode='"+city_postcode+"';")
    result = cur.fetchone()
    city_id_array = np.append(city_id_array, result[0])





In [307]:
conn.commit()
conn.close()

In [308]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [309]:
for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #address_table
 
    city_id_fk = str(int(city_id_array[idx]))


    sql_fill_address = "INSERT INTO gta_p8.address (address_street, address_number, city_id) SELECT '"+address_street+"', '"+address_number+"', '"+city_id_fk+"';"
  
    cur.execute(sql_fill_city, (address_street, address_number, city_id_fk))




In [310]:
conn.commit()
conn.close()

In [311]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [312]:
for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:





In [313]:
conn.commit()
conn.close()

## Restaurants

In [314]:
for idx, row in restaurant_zurich.iterrows():
    #prepare input values
    restaurant_name = row['name'] #<class 'str'>

    restaurant_website = row['website'] #<class 'str'>
    if(type(restaurant_website) == float):
        restaurant_website = 'keine Website vorhanden!'

    restaurant_type = row['cuisine']
    if(type(restaurant_type) == float):
        restaurant_type = 'keine Kategorie!'

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'   

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>
    